In [1]:
import torch
import torch.nn as nn
from torch._dynamo import OptimizedModule
from acvl_utils.cropping_and_padding.padding import pad_nd_image
from typing import Tuple, Union, List, Optional
import numpy as np
from tqdm import tqdm
import itertools

import SimpleITK as sitk
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import os
import json
from batchgenerators.utilities.file_and_folder_operations import join, save_pickle
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO

In [ ]:
def load_json(file: str):
    with open(file, 'r') as f:
        a = json.load(f)
    return a

In [ ]:
print(torch.__version__)

2.3.0+cpu


In [ ]:
checkpoint = torch.load(r'U:\YCZhang\Dissertation\data\.bash_history\fold_0\medsam_model_best.pth',
                                    map_location=torch.device('cpu'))

In [ ]:
checkpoint.keys()

dict_keys(['model', 'optimizer', 'epoch'])

In [ ]:
model = checkpoint['model']
model

OrderedDict([('image_encoder.pos_embed',
              tensor([[[[-0.0764, -0.0766, -0.0758,  ...,  0.8678,  0.8556,  0.8849],
                        [-0.0391, -0.0660, -0.0636,  ...,  0.8966,  0.8738,  0.8970],
                        [ 0.0330,  0.0182,  0.0210,  ...,  0.8951,  0.8770,  0.8877],
                        ...,
                        [ 0.3421, -0.1760, -0.6143,  ...,  0.8960,  0.8891,  0.8792],
                        [ 0.4352, -0.1042, -0.5758,  ...,  0.8828,  0.8826,  0.8935],
                        [ 0.4933, -0.0760, -0.5824,  ...,  0.8815,  0.8946,  0.8846]],
              
                       [[-0.0961, -0.0773, -0.0699,  ...,  0.8872,  0.8857,  0.8863],
                        [-0.0481, -0.0675, -0.0604,  ...,  0.9004,  0.9019,  0.8918],
                        [ 0.0373,  0.0201,  0.0220,  ...,  0.8942,  0.9001,  0.8897],
                        ...,
                        [ 0.3390, -0.1770, -0.6156,  ...,  0.8937,  0.8821,  0.8885],
                        [

In [ ]:
for key, value in model.items():
    print(key, value.shape)

image_encoder.pos_embed torch.Size([1, 64, 64, 768])
image_encoder.patch_embed.proj.weight torch.Size([768, 3, 16, 16])
image_encoder.patch_embed.proj.bias torch.Size([768])
image_encoder.blocks.0.norm1.weight torch.Size([768])
image_encoder.blocks.0.norm1.bias torch.Size([768])
image_encoder.blocks.0.attn.rel_pos_h torch.Size([27, 64])
image_encoder.blocks.0.attn.rel_pos_w torch.Size([27, 64])
image_encoder.blocks.0.attn.qkv.weight torch.Size([2304, 768])
image_encoder.blocks.0.attn.qkv.bias torch.Size([2304])
image_encoder.blocks.0.attn.proj.weight torch.Size([768, 768])
image_encoder.blocks.0.attn.proj.bias torch.Size([768])
image_encoder.blocks.0.norm2.weight torch.Size([768])
image_encoder.blocks.0.norm2.bias torch.Size([768])
image_encoder.blocks.0.mlp.lin1.weight torch.Size([3072, 768])
image_encoder.blocks.0.mlp.lin1.bias torch.Size([3072])
image_encoder.blocks.0.mlp.lin2.weight torch.Size([768, 3072])
image_encoder.blocks.0.mlp.lin2.bias torch.Size([768])
image_encoder.blocks.

In [ ]:
raw_1 = r'U:\YCZhang\Dissertation\data\.bash_history\BRATS_314.nii.gz'